In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=104745/Rafael-Nadal&table=mcp-serve'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='mcp-serve')
print(table.prettify())

driver.quit()


<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="mcp-serve">
 <thead>
  <tr>
   <th align="left">
    <span title="">
     Match
    </span>
   </th>
   <th align="left">
    <span title="">
     Result
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of serves that
were unreturned">
     Unret%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of service points
won on either the serve or second shot">
     &lt;=3 W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of points won when
return was put in play">
     RiP W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Serve Impact: Advanced stat
estimating how many service points
were won due to the serve">
     SvImpact
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of first serves
that were unreturned">
     1st: Unret%
    </span>
   </th>
   <th a

In [2]:
data = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            data.append((cell.get_text(strip=True)))


2024 Davis Cup RR
L vsVan De Zandschulp
13.2%
34.2%
53.1%
26.8%
15.0%
55.0%
64.7%
41.0%
36.4%
77.8%
-
11.8%
11.8%
40.0%
4
2024 Paris Olympics R32
L vsDjokovic
13.8%
22.4%
41.7%
20.5%
16.7%
23.8%
45.7%
26.0%
15.8%
56.5%
62.5%
7.1%
21.4%
30.8%
-49
2024 Paris Olympics R64
W vsFucsovics
15.2%
31.5%
62.7%
26.2%
19.1%
38.2%
61.8%
34.0%
16.1%
62.2%
85.7%
4.8%
14.3%
65.0%
-72
2024 Bastad F
L vsBorges
12.5%
21.4%
39.1%
17.3%
15.8%
23.7%
43.8%
22.9%
35.3%
57.1%
100.0%
6.7%
20.0%
28.6%
-57
2024 Bastad QF
W vsNavone
13.2%
21.5%
48.5%
18.0%
16.5%
26.6%
53.0%
23.8%
24.3%
54.8%
57.1%
7.9%
13.2%
40.0%
-39
2024 Bastad R32
W vsBorg
30.2%
45.3%
67.6%
47.0%
35.9%
53.8%
72.0%
58.7%
20.0%
75.0%
-
14.3%
21.4%
58.3%
-79
2024 Roland Garros R128
L vsZverev
11.7%
20.7%
50.5%
20.0%
12.8%
25.6%
51.5%
24.6%
15.2%
60.0%
58.3%
10.0%
10.0%
48.1%
-18
2024 Rome R64
L vsHurkacz
15.1%
20.8%
45.5%
19.2%
18.4%
23.7%
41.9%
24.2%
26.3%
57.9%
75.0%
7.1%
14.3%
53.8%
-45
2024 Madrid R64
W vsDe Minaur
8.1%
17.7%
65.5%
13.1%
6.7%


In [3]:
data = [d.replace('\xa0',' ') for d in data]

In [4]:
year_tourney_round = data[::17] # year tournament round in a single string

In [5]:
df_rows = []
for i in range(0, len(data)):
    if i % 17 == 0:
        row = data[i:i+17]
        df_rows.append(row)

In [6]:
cols = ['year tourney round', 'result opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%',
        '1st: <=3 W%', '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%', '2nd: Unret%', '2nd: <=3 W%',
        '2nd: RiP W%', '2ndAgg']
df = pd.DataFrame(df_rows, columns=cols)
df['year'] = df['year tourney round'].apply(lambda x: x.split(' ')[0])
df['tourney'] = df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
df['round'] = df['year tourney round'].apply(lambda x: x.split(' ')[-1]) 
df['result'] = df['result opponent'].apply(lambda x: x.split(' ')[0])
df['opponent'] = df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
df.head(10)

,overall: Unret%,overall: <=3 W%,overall: RiP W%,overall: SvImpact,1st: Unret%,1st: <=3 W%,1st: RiP W%,1st: SvImpact,1st: D Wide%,1st: A Wide%,1st: A BP Wide%,2nd: Unret%,2nd: <=3 W%,2nd: RiP W%,2ndAgg,year,tourney,round,result,opponent
0,13.2%,34.2%,53.1%,26.8%,15.0%,55.0%,64.7%,41.0%,36.4%,77.8%,-,11.8%,11.8%,40.0%,4,2024,Davis Cup,RR,L,Van
1,13.8%,22.4%,41.7%,20.5%,16.7%,23.8%,45.7%,26.0%,15.8%,56.5%,62.5%,7.1%,21.4%,30.8%,-49,2024,Paris Olympics,R32,L,Djokovic
2,15.2%,31.5%,62.7%,26.2%,19.1%,38.2%,61.8%,34.0%,16.1%,62.2%,85.7%,4.8%,14.3%,65.0%,-72,2024,Paris Olympics,R64,W,Fucsovics
3,12.5%,21.4%,39.1%,17.3%,15.8%,23.7%,43.8%,22.9%,35.3%,57.1%,100.0%,6.7%,20.0%,28.6%,-57,2024,Bastad,F,L,Borges
4,13.2%,21.5%,48.5%,18.0%,16.5%,26.6%,53.0%,23.8%,24.3%,54.8%,57.1%,7.9%,13.2%,40.0%,-39,2024,Bastad,QF,W,Navone
5,30.2%,45.3%,67.6%,47.0%,35.9%,53.8%,72.0%,58.7%,20.0%,75.0%,-,14.3%,21.4%,58.3%,-79,2024,Bastad,R32,W,Borg
6,11.7%,20.7%,50.5%,20.0%,12.8%,25.6%,51.5%,24.6%,15.2%,60.0%,58.3%,10.0%,10.0%,48.1%,-18,2024,Roland Garros,R128,L,Zverev
7,15.1%,20.8%,45.5%,19.2%,18.4%,23.7%,41.9%,24.2%,26.3%,57.9%,75.0%,7.1%,14.3%,53.8%,-45,2024,Rome,R64,L,Hurkacz
8,8.1%,17.7%,65.5%,13.1%,6.7%,17.8%,69.0%,13.6%,10.0%,40.0%,50.0%,13.3%,20.0%,53.8%,11,2024,Madrid,R64,W,De
9,18.4%,30.6%,67.6%,22.7%,25.0%,34.4%,70.8%,31.6%,0.0%,73.7%,100.0%,7.1%,28.6%,61.5%,-53,2024,Barcelona,R64,W,Cobolli


In [7]:
neworder = ['year','tourney', 'round', 'result', 'opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%', '1st: <=3 W%',
       '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%',
       '2nd: Unret%', '2nd: <=3 W%', '2nd: RiP W%', '2ndAgg']

df = df[neworder]

In [8]:

df.to_csv('../data/nadal_serve_chart.csv', index=False) #save to csv in data folder

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=104745/Rafael-Nadal&table=serve-speed'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='serve-speed')
print(table.prettify())

driver.quit()

<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="serve-speed">
 <thead>
  <tr>
   <th align="left">
    Match
   </th>
   <th align="left">
    Result
   </th>
   <th align="right" class="header">
    <span title="Average serve speed">
     Avg Speed
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average 1st serve speed">
     1st Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Standard deviation of
1st serve speed">
     1st StDev
    </span>
   </th>
   <th align="right" class="header">
    <span t""="" title="Average speed of 1st serves
down the ">
     1st T Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average speed of
1st serves hit wide">
     1st Wide Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Fastest second serve">
     Max 1st
    </span>
   </th>
   <th align="right" class="header">
    <span title="Slowest second serve">
  

In [10]:
speed = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            speed.append((cell.get_text(strip=True)))


2022 US Open R16
L vsTiafoe
108.5
113.9
7.7
116.9
112.5
123
77
101.2
4.1
101.8
100.3
108
92
2022 US Open R32
W vsGasquet
107.9
112.5
6.3
117.2
108.0
122
103
99.3
5.1
99.6

109
90
2022 US Open R64
W vsFognini
106.5
112.2
7.9
114.0
108.4
122
76
97.3
4.6
97.6
95.0
103
79
2022 US Open R128
W vsHijikata
107.2
113.8
5.8
117.0
107.8
122
101
98.2
3.6
96.9

105
93
2022 Wimbledon QF
W vsFritz
103.0
107.0
6.5
109.2
103.6
120
95
94.7
5.4
94.7
94.0
112
83
2022 Wimbledon R16
W vsVan De Zandschulp
109.9
115.1
4.5
118.5
111.4
123
105
100.1
4.7
100.1

109
89
2022 Wimbledon R32
W vsSonego
110.8
115.0
5.1
118.1
112.2
124
105
100.4
4.6
99.8
98.0
107
89
2022 Wimbledon R64
W vsBerankis
107.6
111.8
5.7
115.0
107.5
122
98
98.9
4.2
98.8
99.0
107
90
2022 Wimbledon R128
W vsCerundolo
109.7
114.3
5.6
116.7
111.4
124
100
98.1
4.9
97.7
104.0
110
91
2019 US Open F
W vsMedvedev
105.9
115.2
6.2
119.0
111.7
125
94
91.8
4.9
91.3
92.7
102
81
2019 US Open SF
W vsBerrettini
108.1
116.8
6.9
119.9
111.1
126
100
96.5
5.2
96.0

In [11]:
speed = [s.replace('\xa0',' ') for s in speed]

In [12]:
year_tourney_round = speed[::15]

In [13]:
speed_df_rows = []
for i in range(0, len(speed)):
    if i % 15 == 0:
        row = speed[i:i+15]
        speed_df_rows.append(row)

In [14]:
cols = ['year tourney round', 'result opponent', 'avg speed', '1st: avg', '1st: stdev', '1st: t avg', '1st: wide avg', '1st: max', 
     '1st: min', '2nd: avg', '2nd: stdev', '2nd: t avg', '2nd: wide avg', '2nd: max', '2nd: min']

speed_df = pd.DataFrame(speed_df_rows, columns=cols)
speed_df['year'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[0])
speed_df['tourney'] = speed_df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
speed_df['round'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[-1])
speed_df['result'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[0])
speed_df['opponent'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
speed_df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
speed_df.head(10)

,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min,year,tourney,round,result,opponent
0,108.5,113.9,7.7,116.9,112.5,123,77,101.2,4.1,101.8,100.3,108,92,2022,US Open,R16,L,Tiafoe
1,107.9,112.5,6.3,117.2,108.0,122,103,99.3,5.1,99.6,,109,90,2022,US Open,R32,W,Gasquet
2,106.5,112.2,7.9,114.0,108.4,122,76,97.3,4.6,97.6,95.0,103,79,2022,US Open,R64,W,Fognini
3,107.2,113.8,5.8,117.0,107.8,122,101,98.2,3.6,96.9,,105,93,2022,US Open,R128,W,Hijikata
4,103.0,107.0,6.5,109.2,103.6,120,95,94.7,5.4,94.7,94.0,112,83,2022,Wimbledon,QF,W,Fritz
5,109.9,115.1,4.5,118.5,111.4,123,105,100.1,4.7,100.1,,109,89,2022,Wimbledon,R16,W,Van
6,110.8,115.0,5.1,118.1,112.2,124,105,100.4,4.6,99.8,98.0,107,89,2022,Wimbledon,R32,W,Sonego
7,107.6,111.8,5.7,115.0,107.5,122,98,98.9,4.2,98.8,99.0,107,90,2022,Wimbledon,R64,W,Berankis
8,109.7,114.3,5.6,116.7,111.4,124,100,98.1,4.9,97.7,104.0,110,91,2022,Wimbledon,R128,W,Cerundolo
9,105.9,115.2,6.2,119.0,111.7,125,94,91.8,4.9,91.3,92.7,102,81,2019,US Open,F,W,Medvedev


In [15]:
neworder = ['year','tourney', 'round', 'result', 'opponent'] + cols[2:]
speed_df = speed_df[neworder]


In [16]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].replace('', np.nan)
speed_df

,year,tourney,round,result,opponent,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min
0,2022,US Open,R16,L,Tiafoe,108.5,113.9,7.7,116.9,112.5,123,77,101.2,4.1,101.8,100.3,108,92
1,2022,US Open,R32,W,Gasquet,107.9,112.5,6.3,117.2,108.0,122,103,99.3,5.1,99.6,NaN,109,90
2,2022,US Open,R64,W,Fognini,106.5,112.2,7.9,114.0,108.4,122,76,97.3,4.6,97.6,95.0,103,79
3,2022,US Open,R128,W,Hijikata,107.2,113.8,5.8,117.0,107.8,122,101,98.2,3.6,96.9,NaN,105,93
4,2022,Wimbledon,QF,W,Fritz,103.0,107.0,6.5,109.2,103.6,120,95,94.7,5.4,94.7,94.0,112,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2011,Australian Open,QF,L,Ferrer,104.0,111.3,7.7,NaN,NaN,128,96,88.0,3.2,NaN,NaN,97,82
156,2011,Australian Open,R16,W,Cilic,108.1,113.6,6.6,NaN,NaN,124,101,86.9,3.7,NaN,NaN,92,79
157,2011,Australian Open,R32,W,Tomic,103.0,112.9,8.5,NaN,NaN,128,90,87.5,4.0,NaN,NaN,97,80
158,2011,Australian Open,R64,W,Sweeting,109.2,118.4,8.4,NaN,NaN,129,103,90.3,3.3,NaN,NaN,97,82


In [17]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].astype(float)

In [18]:
speed_df.to_csv('../data/nadal_serve_speed.csv', index=False) #change the name of the file to save it as you want